- **find_element**
    - https://hyunsooworld.tistory.com/entry/%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%EC%98%A4%EB%A5%98-AttributeError-WebDriver-object-has-no-attribute-findelementbycssselector-%EC%98%A4%EB%A5%98%ED%95%B4%EA%B2%B0

In [ ]:
# !pip install selenium

### Import

In [1]:
import time
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

### 크롬 브라우저

In [2]:
def Open_chrome(search:str):
    
    ''' 📢 원하는 검색어 입력해서 크롬 브라우저 열기 '''
    
    url = "https://map.naver.com/v5/search/" + search

    driver = webdriver.Chrome()
    driver.get(url)

    # iframe
    iframe_element = driver.find_element(By.ID, "searchIframe")
    driver.switch_to.frame(iframe_element)
    
    return driver

### 스크롤 조정

In [3]:
def scroll_container_by_amount(driver, container_element):
    
    ''' 📢 스크롤 끝까지 내리기 '''
    
    prev_scroll_height = 0
    
    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", container_element)
        time.sleep(1.5)
        
        curr_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", container_element)
        
        if curr_scroll_height == prev_scroll_height:
            break
        
        prev_scroll_height = curr_scroll_height

### 크롤링

In [4]:
def Navermap_crawling(search:str, ele_tag:str, name_tag:str, catg_tag:str):
    
    """ 📢 네이버 지도의 마지막 페이지까지 크롤링 """
    
    driver = Open_chrome(search)
    
    df = pd.DataFrame()
    previous_text = None  

    while True:
        now_page = driver.find_element(By.CSS_SELECTOR, f"#app-root > div > div.XUrfU > div.zRM9F > a.mBN2s.qxokY")
        current_text = now_page.text

        if previous_text != current_text:
            container_element = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
            scroll_container_by_amount(driver, container_element)

            # find_elements(검색어에 따라 변경 필요) : 상호명 / 카테고리
            element = driver.find_elements(By.CSS_SELECTOR, ele_tag)
            name = [i.find_element(By.CSS_SELECTOR, name_tag).text for i in element]
            category = [i.find_element(By.CSS_SELECTOR, catg_tag).text for i in element]
            
            print('현재 페이지:', current_text)
            print(name)
            
            new_data = {'Name': name, 'Category': category, 'Search': search}
            df = df.append(pd.DataFrame(new_data), ignore_index=True)
            df.to_excel(f'./data/{search}_navermap_크롤링.xlsx', index=False)
            
            # 다음 페이지 클릭
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7) > svg").click()

        else:
            break

        previous_text = current_text
        
    return df

In [8]:
search = "강남 카페"
element_tag = "#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0 > div > div"
name_tag = "span.place_bluelink.TYaxT"
category_tag = "span.KCMnt"

result_df = Navermap_crawling(search, element_tag, name_tag, category_tag)
result_df

현재 페이지: 1
['아방베이커리 센터필드점', '소쿠리베니스', 'SADDLER HAUS', '루비스 카페', '수플레 팬케이크 소과당 본점', '뉴욕라츠오베이글스', '런던 베이글 뮤지엄 도산점', '트키 도산', '알베르', '마일스톤 커피', '쇼토', '카페무니 압구정본점', '클로리스 코엑스몰점', '1209', '까치화방 강남역점', '더달달', '라브리크 서울', '서울북카페 욕망의북카페', '정월', '호두까끼형제', '쿠스 카페', '올드페리도넛 가로수길점', '가배도 신논현', '도산맘마미아', '스크렘', '비파티세리', '세시셀라 도산공원점', '젠제로', '바게트케이', '노티드 삼성', '이디야커피랩', '마니달오 선릉본점', '고센인코퍼레이티드 청담점', '이치서울']
현재 페이지: 2
['플디 도산점', '브레댄코 강남대치점', '카멜커피 도산점', '나따오비까 신사점', '포스톤즈 삼성본점', '플라워베이커리 가로수길점', '구테로이테', '듀자미', '로젯런던베이글', '우디집 도산점', '케이코스랜드', '쿠라우니', '설빙 강남역점', '빙빙빙', '젠틀커피', '% ARABICA 스타필드 코엑스몰점', '아가젤라또', '테이블스', '카페 라미스콘 청담점', '쁠로13', '블레스롤 강남논현점', '하이테이블 차병원사거리점', '바빌리안테이블', '가배도 코엑스점', '껠끄쇼즈', '빵어니스타 압구정점', '왕가탕후루 강남역점', '크리에잇쿠키 역삼본점', '프롬헤라스 도산 플래그십 스토어', '엔커피스튜디오']
현재 페이지: 3
['몽크투바흐', '애니오케이션 카페', '티하우스 청담', '에그서울', '브라이언스커피', '아우어베이커리 가로수길점', '블루보틀 역삼 카페', '마일드하이클럽', '타르틴베이커리 도산점', '겟썸커피 Downstairs', '스테이지 바이 고디바', '파이하우스', '더죠이 코엑스몰점', '버터풀앤크리멀러스 압구정 본점', '리암스 케이커리', '크림라벨 신사점', '읍천리382 압구정 

,Name,Category,Search
0,아방베이커리 센터필드점,베이커리,강남 카페
1,소쿠리베니스,카페,강남 카페
2,SADDLER HAUS,와플,강남 카페
3,루비스 카페,"카페,디저트",강남 카페
4,수플레 팬케이크 소과당 본점,"카페,디저트",강남 카페
...,...,...,...
179,멜로드 도산,케이크전문,강남 카페
180,벤스쿠키 가로수길점,베이커리,강남 카페
181,카페 두레브 세곡점,카페,강남 카페
182,퀸즈파크 청담점,브런치,강남 카페


In [7]:
search = "홍대 옷"
element_tag = "#_pcmap_list_scroll_container > ul > li > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1) > div > div"
name_tag = "span.place_bluelink.YwYLL"
category_tag = "span.YzBgS"

result_df = Navermap_crawling(search, element_tag, name_tag, category_tag)
result_df

현재 페이지: 1
['울트라빈티지', '커먼유니크 쇼룸', '도쿄유즈드 홍대', '무신사 스탠다드 홍대', '널디 플래그쉽스토어 서울', '하이츠스토어', '웍스아웃 홍대 라이즈점', '젠틀몬스터 홍대 플래그십스토어', '프라이탁 홍대점', 'ABC마트 GS 홍대점', '공구우먼 합정점', '호텐', '슈펜 홍대점', '예스아이씨', '아더 홍대 스페이스', '모드맨', '에버닌스테어스', '에이랜드 홍대점', '커먼유니크 쇼룸', '시공간 연남점', '미니스피어싱 홍대점', '빈티지샵 페이지원', '젝시믹스 핏 스토어 홍대점', '드림윙즈', '미니스피어싱 홍대2호점', 'AAPLACE', '로미스토리 플래그십스토어 홍대점', '리터치', '칼하트WIP 홍대와우산점', '로스트가든', '게릴라라디오']
현재 페이지: 2
['하바티', '미래의 미래', '풋락커 홍대점', '스페이스모리스', '핑크에이지 홍대본점', '탑텐 홍대점', '브라샵로라 홍대점', '울트라빈티지', '어나더어썸 홍대점', '바르카', '로맨틱무브 홍대본점', '힐링햇 홍대점', '스윙잉서울 홍대점', '미로미로 홍대점', '아베르하우스', '루카', '필악세사리', '빈티지산타', '라이커샵', '에잇스톤스', '리틀리들', 'ABC마트 SP 홍대점', '빈티지바이로웰', '해피히피', '하이마운틴 빈티지', '에메필 홍대2호점', 'LMC HONGDAE FLAGSHIP STORE', '아홀로틀실버']
현재 페이지: 3
['가발나라 홍대점', '와우빈티지 홍대점', '교복몰 홍대점', 'ABC마트 GS 홍대홍익로점', '리슬 홍대점', '시공간 홍대본점', '챔피온스토어 홍대AK&점', '한스룸 연남', '인그루브', '닥터마틴 홍대점', '피치비 위그', '탑텐 홍대중앙점', '주코빈티지', '오늘의 빈티지', '로서울', 'Lee 홍대플래그쉽스토어', '유티에이서울', '잠자는라푼젤', '마리떼프랑소와저버 홍대점', '부기홀리데이', '쥬니쥬 쇼룸 연남점', '엔드 스트릿', '샘

,Name,Category,Search
0,울트라빈티지,구제의류,홍대 옷
1,커먼유니크 쇼룸,여성의류,홍대 옷
2,도쿄유즈드 홍대,구제의류,홍대 옷
3,무신사 스탠다드 홍대,종합패션,홍대 옷
4,널디 플래그쉽스토어 서울,패션,홍대 옷
...,...,...,...
151,밀라튜튜,유아동복,홍대 옷
152,탱고슈즈 마라비샤,신발,홍대 옷
153,데일리윙,남성의류,홍대 옷
154,에이미컴퍼니,보세의류,홍대 옷
